In [1]:
import json
import requests

import pandas as pd 

In [32]:
def json_to_df(data):
    elements = data['elements']
    places = {'TIPO': [], 'LATITUD': [], 'LONGITUD': [], 'NOMBRE LOCAL': [], 'DIRECCION': []}
    
    for i in elements:
        
        tipo = i.get('tags', None).get('amenity', None)
        latitude = i.get('lat', None)
        longitude = i.get('lon', None)
        name = i.get('tags', {}).get('name', "NO NAME")
        street = i.get('tags', {}).get('addr:street', "NO STREET")
        number = i.get('tags', {}).get('addr:housenumber', 9999)

        places['TIPO'].append(tipo)
        places['LATITUD'].append(latitude)
        places['LONGITUD'].append(longitude)
        places['NOMBRE LOCAL'].append(name)
        places['DIRECCION'].append(street + ' ' + str(number))

            
    return pd.DataFrame(places)

In [3]:
tipo = ["restaurant", "bar", "pub", "cafe", "cinema", "theatre"]
barcelona = (41.33,2.06,41.46,2.27)
madrid = (40.256,-3.995,40.543,-3.306)
bilbao = (43.199,-3.069,43.373,-2.828)
valencia = (39.408,-0.4758,39.548,-0.273)
malaga = (36.658,-4.501,36.754,-4.351)
sevilla = (37.332,-6.094,37.439,-5.893)

In [33]:
def tablas(nodes, location, col):
    """Con esta función se obtienen los locales para los nodes definidos en la localización (marcada por 4 puntos), que traen los argumentos."""

    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    node["amenity"]{location};
    out;
    """
            
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    data = response.json() 
    df_total = json_to_df(data)
    
    for ii,k in enumerate(nodes):

        if ii==0:
            df = df_total[df_total["TIPO"] == k]
        else:
            df = pd.concat([df, df_total[df_total["TIPO"] == k]])

    df.reset_index(inplace=True)
    df.drop(columns="index", inplace=True)
    df["LOCALIDAD"] = col

    print("Se ha obtenido un dataframe con shape:", df.shape)

    return df

In [34]:
df_barcelona = tablas(tipo, barcelona, "Barcelona")

Se ha obtenido un dataframe con shape: (5797, 6)


In [35]:
df_madrid = tablas(tipo, madrid, "Madrid")

Se ha obtenido un dataframe con shape: (8546, 6)


In [38]:
df_bilbao = tablas(tipo, bilbao, "Bilbao")

Se ha obtenido un dataframe con shape: (2481, 6)


In [40]:
df_valencia = tablas(tipo, valencia, "Valencia")

Se ha obtenido un dataframe con shape: (1688, 6)


In [41]:
df_malaga = tablas(tipo, malaga, "Málaga")

Se ha obtenido un dataframe con shape: (610, 6)


In [42]:
df_sevilla = tablas(tipo, sevilla, "Sevilla")

Se ha obtenido un dataframe con shape: (1861, 6)


In [43]:
df_locales = pd.concat([df_barcelona,df_madrid,df_bilbao,df_valencia,df_malaga,df_sevilla],axis=0)

df_locales.reset_index(inplace=True)
df_locales.drop(columns="index", inplace=True)
df_locales

,TIPO,LATITUD,LONGITUD,NOMBRE LOCAL,DIRECCION,LOCALIDAD
0,restaurant,41.381687,2.076645,Ateneu de Sant Just Desvern,NO STREET 9999,Barcelona
1,restaurant,41.383643,2.158303,Amaltea,NO STREET 9999,Barcelona
2,restaurant,41.389518,2.162177,Hanibishi,NO STREET 9999,Barcelona
3,restaurant,41.389997,2.131837,Moncho's House,NO STREET 9999,Barcelona
4,restaurant,41.383672,2.180639,Taller de Tapas,l'Argenteria 51,Barcelona
...,...,...,...,...,...,...
20978,theatre,37.385602,-6.003795,Casalá Teatro,NO STREET 9999,Sevilla
20979,theatre,37.383316,-6.000608,Teatro Flamenco Triana,NO STREET 9999,Sevilla
20980,theatre,37.401108,-5.988291,La sala,NO STREET 9999,Sevilla
20981,theatre,37.367891,-5.960394,Teatro Salvador Távora,8 9999,Sevilla


In [44]:
df_locales.loc[df_locales["DIRECCION"]=="NO STREET 9999", "DIRECCION"] = ""
df_locales

,TIPO,LATITUD,LONGITUD,NOMBRE LOCAL,DIRECCION,LOCALIDAD
0,restaurant,41.381687,2.076645,Ateneu de Sant Just Desvern,,Barcelona
1,restaurant,41.383643,2.158303,Amaltea,,Barcelona
2,restaurant,41.389518,2.162177,Hanibishi,,Barcelona
3,restaurant,41.389997,2.131837,Moncho's House,,Barcelona
4,restaurant,41.383672,2.180639,Taller de Tapas,l'Argenteria 51,Barcelona
...,...,...,...,...,...,...
20978,theatre,37.385602,-6.003795,Casalá Teatro,,Sevilla
20979,theatre,37.383316,-6.000608,Teatro Flamenco Triana,,Sevilla
20980,theatre,37.401108,-5.988291,La sala,,Sevilla
20981,theatre,37.367891,-5.960394,Teatro Salvador Távora,8 9999,Sevilla


In [45]:
df_locales.rename(columns={"LOCALIDAD":"CIUDAD"}, inplace=True)
df_locales

,TIPO,LATITUD,LONGITUD,NOMBRE LOCAL,DIRECCION,CIUDAD
0,restaurant,41.381687,2.076645,Ateneu de Sant Just Desvern,,Barcelona
1,restaurant,41.383643,2.158303,Amaltea,,Barcelona
2,restaurant,41.389518,2.162177,Hanibishi,,Barcelona
3,restaurant,41.389997,2.131837,Moncho's House,,Barcelona
4,restaurant,41.383672,2.180639,Taller de Tapas,l'Argenteria 51,Barcelona
...,...,...,...,...,...,...
20978,theatre,37.385602,-6.003795,Casalá Teatro,,Sevilla
20979,theatre,37.383316,-6.000608,Teatro Flamenco Triana,,Sevilla
20980,theatre,37.401108,-5.988291,La sala,,Sevilla
20981,theatre,37.367891,-5.960394,Teatro Salvador Távora,8 9999,Sevilla


In [47]:
df_locales.loc[df_locales["CIUDAD"]=="Barcelona", "COMUNIDAD AUTONOMA"] = "Cataluña"
df_locales.loc[df_locales["CIUDAD"]=="Barcelona", "PROVINCIA"] = "Barcelona"

df_locales.loc[df_locales["CIUDAD"]=="Madrid", "COMUNIDAD AUTONOMA"] = "Madrid"
df_locales.loc[df_locales["CIUDAD"]=="Madrid", "PROVINCIA"] = "Madrid"

df_locales.loc[df_locales["CIUDAD"]=="Valencia", "COMUNIDAD AUTONOMA"] = "Valencia"
df_locales.loc[df_locales["CIUDAD"]=="Valencia", "PROVINCIA"] = "Valencia"

df_locales.loc[df_locales["CIUDAD"]=="Bilbao", "COMUNIDAD AUTONOMA"] = "País Vasco"
df_locales.loc[df_locales["CIUDAD"]=="Bilbao", "PROVINCIA"] = "Vizcaya"

df_locales.loc[df_locales["CIUDAD"]=="Málaga", "COMUNIDAD AUTONOMA"] = "Andalucía"
df_locales.loc[df_locales["CIUDAD"]=="Málaga", "PROVINCIA"] = "Málaga"

df_locales.loc[df_locales["CIUDAD"]=="Sevilla", "COMUNIDAD AUTONOMA"] = "Andalucía"
df_locales.loc[df_locales["CIUDAD"]=="Sevilla", "PROVINCIA"] = "Sevilla"


df_locales

,TIPO,LATITUD,LONGITUD,NOMBRE LOCAL,DIRECCION,CIUDAD,COMUNIDAD AUTONOMA,PROVINCIA
0,restaurant,41.381687,2.076645,Ateneu de Sant Just Desvern,,Barcelona,Cataluña,Barcelona
1,restaurant,41.383643,2.158303,Amaltea,,Barcelona,Cataluña,Barcelona
2,restaurant,41.389518,2.162177,Hanibishi,,Barcelona,Cataluña,Barcelona
3,restaurant,41.389997,2.131837,Moncho's House,,Barcelona,Cataluña,Barcelona
4,restaurant,41.383672,2.180639,Taller de Tapas,l'Argenteria 51,Barcelona,Cataluña,Barcelona
...,...,...,...,...,...,...,...,...
20978,theatre,37.385602,-6.003795,Casalá Teatro,,Sevilla,Andalucía,Sevilla
20979,theatre,37.383316,-6.000608,Teatro Flamenco Triana,,Sevilla,Andalucía,Sevilla
20980,theatre,37.401108,-5.988291,La sala,,Sevilla,Andalucía,Sevilla
20981,theatre,37.367891,-5.960394,Teatro Salvador Távora,8 9999,Sevilla,Andalucía,Sevilla


In [57]:
df_locales["USUARIO DE EMPRESA"] = ""
df_locales["EMAIL"] = ""
df_locales["TELEFONO"] = ""
df_locales["ACUM VERDE"] = ""
df_locales["ACUM AMARILLO"] = ""
df_locales["ACUM ROJO"] = ""
df_locales["VALORACIONES"] = ""
df_locales["CERTIFICACION"] = ""
df_locales["IMAGEN"] = ""

df_locales

,TIPO,LATITUD,LONGITUD,NOMBRE LOCAL,DIRECCION,CIUDAD,COMUNIDAD AUTONOMA,PROVINCIA,USUARIO DE EMPRESA,EMAIL,TELEFONO,ACUM VERDE,ACUM AMARILLO,ACUM ROJO,VALORACIONES,CERTIFICACION,SECTOR,IMAGEN
0,restaurante,41.381687,2.076645,Ateneu de Sant Just Desvern,,Barcelona,Cataluña,Barcelona,,,,,,,,,hostelería,
1,restaurante,41.383643,2.158303,Amaltea,,Barcelona,Cataluña,Barcelona,,,,,,,,,hostelería,
2,restaurante,41.389518,2.162177,Hanibishi,,Barcelona,Cataluña,Barcelona,,,,,,,,,hostelería,
3,restaurante,41.389997,2.131837,Moncho's House,,Barcelona,Cataluña,Barcelona,,,,,,,,,hostelería,
4,restaurante,41.383672,2.180639,Taller de Tapas,l'Argenteria 51,Barcelona,Cataluña,Barcelona,,,,,,,,,hostelería,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20978,teatro,37.385602,-6.003795,Casalá Teatro,,Sevilla,Andalucía,Sevilla,,,,,,,,,ocio,
20979,teatro,37.383316,-6.000608,Teatro Flamenco Triana,,Sevilla,Andalucía,Sevilla,,,,,,,,,ocio,
20980,teatro,37.401108,-5.988291,La sala,,Sevilla,Andalucía,Sevilla,,,,,,,,,ocio,
20981,teatro,37.367891,-5.960394,Teatro Salvador Távora,8 9999,Sevilla,Andalucía,Sevilla,,,,,,,,,ocio,


In [54]:
df_locales.loc[df_locales["TIPO"]=="restaurant", "TIPO"] = "restaurante"
df_locales.loc[df_locales["TIPO"]=="bar", "TIPO"] = "bar"
df_locales.loc[df_locales["TIPO"]=="pub", "TIPO"] = "pub"
df_locales.loc[df_locales["TIPO"]=="cafe", "TIPO"] = "cafe"
df_locales.loc[df_locales["TIPO"]=="cinema", "TIPO"] = "cine"
df_locales.loc[df_locales["TIPO"]=="theatre", "TIPO"] = "teatro"


df_locales

,TIPO,LATITUD,LONGITUD,NOMBRE LOCAL,DIRECCION,CIUDAD,COMUNIDAD AUTONOMA,PROVINCIA,USUARIO DE EMPRESA,EMAIL,TELEFONO,ACUM VERDE,ACUM AMARILLO,ACUM ROJO,VALORACIONES,CERTIFICACION
0,restaurante,41.381687,2.076645,Ateneu de Sant Just Desvern,,Barcelona,Cataluña,Barcelona,,,,,,,,
1,restaurante,41.383643,2.158303,Amaltea,,Barcelona,Cataluña,Barcelona,,,,,,,,
2,restaurante,41.389518,2.162177,Hanibishi,,Barcelona,Cataluña,Barcelona,,,,,,,,
3,restaurante,41.389997,2.131837,Moncho's House,,Barcelona,Cataluña,Barcelona,,,,,,,,
4,restaurante,41.383672,2.180639,Taller de Tapas,l'Argenteria 51,Barcelona,Cataluña,Barcelona,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20978,teatro,37.385602,-6.003795,Casalá Teatro,,Sevilla,Andalucía,Sevilla,,,,,,,,
20979,teatro,37.383316,-6.000608,Teatro Flamenco Triana,,Sevilla,Andalucía,Sevilla,,,,,,,,
20980,teatro,37.401108,-5.988291,La sala,,Sevilla,Andalucía,Sevilla,,,,,,,,
20981,teatro,37.367891,-5.960394,Teatro Salvador Távora,8 9999,Sevilla,Andalucía,Sevilla,,,,,,,,


In [55]:
df_locales.loc[(df_locales["TIPO"]!="cine") & (df_locales["TIPO"]!="teatro"), "SECTOR"] = "hostelería"
df_locales.loc[(df_locales["TIPO"]=="cine") | (df_locales["TIPO"]=="teatro"), "SECTOR"] = "ocio"


df_locales

,TIPO,LATITUD,LONGITUD,NOMBRE LOCAL,DIRECCION,CIUDAD,COMUNIDAD AUTONOMA,PROVINCIA,USUARIO DE EMPRESA,EMAIL,TELEFONO,ACUM VERDE,ACUM AMARILLO,ACUM ROJO,VALORACIONES,CERTIFICACION,SECTOR
0,restaurante,41.381687,2.076645,Ateneu de Sant Just Desvern,,Barcelona,Cataluña,Barcelona,,,,,,,,,hostelería
1,restaurante,41.383643,2.158303,Amaltea,,Barcelona,Cataluña,Barcelona,,,,,,,,,hostelería
2,restaurante,41.389518,2.162177,Hanibishi,,Barcelona,Cataluña,Barcelona,,,,,,,,,hostelería
3,restaurante,41.389997,2.131837,Moncho's House,,Barcelona,Cataluña,Barcelona,,,,,,,,,hostelería
4,restaurante,41.383672,2.180639,Taller de Tapas,l'Argenteria 51,Barcelona,Cataluña,Barcelona,,,,,,,,,hostelería
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20978,teatro,37.385602,-6.003795,Casalá Teatro,,Sevilla,Andalucía,Sevilla,,,,,,,,,ocio
20979,teatro,37.383316,-6.000608,Teatro Flamenco Triana,,Sevilla,Andalucía,Sevilla,,,,,,,,,ocio
20980,teatro,37.401108,-5.988291,La sala,,Sevilla,Andalucía,Sevilla,,,,,,,,,ocio
20981,teatro,37.367891,-5.960394,Teatro Salvador Távora,8 9999,Sevilla,Andalucía,Sevilla,,,,,,,,,ocio


In [58]:
df_locales.columns

Index(['TIPO', 'LATITUD', 'LONGITUD', 'NOMBRE LOCAL', 'DIRECCION', 'CIUDAD',
       'COMUNIDAD AUTONOMA', 'PROVINCIA', 'USUARIO DE EMPRESA', 'EMAIL',
       'TELEFONO', 'ACUM VERDE', 'ACUM AMARILLO', 'ACUM ROJO', 'VALORACIONES',
       'CERTIFICACION', 'SECTOR', 'IMAGEN'],
      dtype='object')

In [60]:
df_locales = df_locales[['TIPO', 'SECTOR', 'LATITUD', 'LONGITUD', 'NOMBRE LOCAL', 'DIRECCION', 'CIUDAD',
       'COMUNIDAD AUTONOMA', 'PROVINCIA', 'USUARIO DE EMPRESA', 'EMAIL',
       'TELEFONO', 'ACUM VERDE', 'ACUM AMARILLO', 'ACUM ROJO', 'VALORACIONES',
       'CERTIFICACION', 'IMAGEN']]

In [61]:
df_locales

,TIPO,SECTOR,LATITUD,LONGITUD,NOMBRE LOCAL,DIRECCION,CIUDAD,COMUNIDAD AUTONOMA,PROVINCIA,USUARIO DE EMPRESA,EMAIL,TELEFONO,ACUM VERDE,ACUM AMARILLO,ACUM ROJO,VALORACIONES,CERTIFICACION,IMAGEN
0,restaurante,hostelería,41.381687,2.076645,Ateneu de Sant Just Desvern,,Barcelona,Cataluña,Barcelona,,,,,,,,,
1,restaurante,hostelería,41.383643,2.158303,Amaltea,,Barcelona,Cataluña,Barcelona,,,,,,,,,
2,restaurante,hostelería,41.389518,2.162177,Hanibishi,,Barcelona,Cataluña,Barcelona,,,,,,,,,
3,restaurante,hostelería,41.389997,2.131837,Moncho's House,,Barcelona,Cataluña,Barcelona,,,,,,,,,
4,restaurante,hostelería,41.383672,2.180639,Taller de Tapas,l'Argenteria 51,Barcelona,Cataluña,Barcelona,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20978,teatro,ocio,37.385602,-6.003795,Casalá Teatro,,Sevilla,Andalucía,Sevilla,,,,,,,,,
20979,teatro,ocio,37.383316,-6.000608,Teatro Flamenco Triana,,Sevilla,Andalucía,Sevilla,,,,,,,,,
20980,teatro,ocio,37.401108,-5.988291,La sala,,Sevilla,Andalucía,Sevilla,,,,,,,,,
20981,teatro,ocio,37.367891,-5.960394,Teatro Salvador Távora,8 9999,Sevilla,Andalucía,Sevilla,,,,,,,,,


In [62]:
df_locales.to_csv("locales.csv")

',TIPO,SECTOR,LATITUD,LONGITUD,NOMBRE LOCAL,DIRECCION,CIUDAD,COMUNIDAD AUTONOMA,PROVINCIA,USUARIO DE EMPRESA,EMAIL,TELEFONO,ACUM VERDE,ACUM AMARILLO,ACUM ROJO,VALORACIONES,CERTIFICACION,IMAGEN\r\n0,restaurante,hostelería,41.3816867,2.0766452,Ateneu de Sant Just Desvern,,Barcelona,Cataluña,Barcelona,,,,,,,,,\r\n1,restaurante,hostelería,41.3836428,2.1583028,Amaltea,,Barcelona,Cataluña,Barcelona,,,,,,,,,\r\n2,restaurante,hostelería,41.3895184,2.1621769,Hanibishi,,Barcelona,Cataluña,Barcelona,,,,,,,,,\r\n3,restaurante,hostelería,41.3899967,2.1318367,Moncho\'s House,,Barcelona,Cataluña,Barcelona,,,,,,,,,\r\n4,restaurante,hostelería,41.3836716,2.1806392,Taller de Tapas,l\'Argenteria 51,Barcelona,Cataluña,Barcelona,,,,,,,,,\r\n5,restaurante,hostelería,41.3942273,2.109899,El Jardí de l\'Abadessa,,Barcelona,Cataluña,Barcelona,,,,,,,,,\r\n6,restaurante,hostelería,41.3975532,2.1230797,Bar Tomás,,Barcelona,Cataluña,Barcelona,,,,,,,,,\r\n7,restaurante,hostelería,41.4523498,2.2087628,La Tagliatella,